# EDA of the NY houses dataset

This notebook explores the NYC houses raw dataset to find out problems and inconsistencies to be addressed during the preprocessing phase

### Import dependencies

In [1]:
import logging

In [2]:
import pandas as pd
import pandas_profiling
import wandb

### Configure logger

In [3]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

### Define constants

In [4]:
PROJECT_NAME = 'nyc_airbnb'
GROUP_NAME = 'eda'
RAW_ARTIFACT_NAME = 'sample.csv:latest'
JOB_TYPE = 'eda'

### Create W&B run and get the raw dataset artifact

In [5]:
run = wandb.init(project=PROJECT_NAME, group=GROUP_NAME, job_type=JOB_TYPE, save_code=True)

wandb: Currently logged in as: tom-ph (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [6]:
raw_dataset_path = wandb.use_artifact(RAW_ARTIFACT_NAME).file()
raw_data_df = pd.read_csv(raw_dataset_path)
logger.info(f'raw dataset shape: {raw_data_df.shape}')

INFO:__main__:raw dataset shape: (20000, 16)


### Apply pandas profiling to explore the data

In [7]:
profile = pandas_profiling.ProfileReport(raw_data_df)
profile.to_widgets()

Summarize dataset:   0%|          | 0/29 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In our opionion, these are the main findings of the exploration:

- Overall **the quality of the data is high.** Almost every feature has no missing values and values ranges seems consistent with expectations. As expected (never say never) the *id* column has no duplicate values. Also the *name* column has ~99% of unique values, and since it is a free text field the duplicate names could refer to distinct houses.
- The
- There are a few outliers with *prices* below the 5 percentile or above the 95 percentile. **During the preprocessing** we will drop the ones above the 95 percentile since they differ a lot from the mean and median and could have a negative effect in the training phase. We will also remove the ones below 10$ because they don't make sense from a business perspective.
- The *last_review* column is a date but it is stored as a text. We will convert this column in the **preprocessing phase**, since in production we will accept only date values for this field.
- There is a significative percentage of missing values in *last_review* and *reviews_per_moth* (~20%). This makes sense, there are houses without reviews. The imputation will be addressed in the **inference pipeline** since it is possible to find this scenario in production.

These are the findings we want to explore further:

- There is a high percentage (~36%) of zeros for the *availability_365* column. We will explore these samples to understand if the zeros make sense from a business point of view or are a replacement for missing values.
- We will also investigate outliers in price to see if the features have similar distributions.

In [8]:
unavailable_df = raw_data_df[raw_data_df['availability_365']==0]
logger.info(f'there are {unavailable_df.shape[0]} records with 0 availability_365')

INFO:__main__:there are 7176 records with 0 availability_365


In [9]:
unavailable_profile = pandas_profiling.ProfileReport(unavailable_df)
unavailable_profile.to_widgets()

Summarize dataset:   0%|          | 0/30 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

The two datasets are really similar dispite the business meaning of the *availability_365* column. It could make sense to impute the zeros and see if the training benefits from this preprocessing strategy.

In [10]:
high_price_df = raw_data_df[raw_data_df['price']>350]
logger.info(f'there are {high_price_df.shape[0]} records with price > 95 percentile')

INFO:__main__:there are 994 records with price > 95 percentile


In [11]:
high_price_profile = pandas_profiling.ProfileReport(high_price_df)
high_price_profile.to_widgets()

Summarize dataset:   0%|          | 0/30 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

The distributions are very similar except for the column *room_type*, that for this portion of data has a high percentage of apartments, and the column *neighbourhood*. This makes sense, as outliers are probably big apartments, or apartments in expensive neighborhoods.

If we drop the outliers we should address the problem of detecting them in production, either with AI or using business knowledge, otherwise predicting them could give inconsistent results.

To improve the results, we should add other variables like house dimensions and number of rooms, and use a feature store for other variables, like the average price per neighbourhood.

### Preprocess data and reapply profiling

In [12]:
# Drop outliers
min_price = 10
max_price = 350
idx = raw_data_df['price'].between(min_price, max_price)
prep_data_df = raw_data_df[idx].copy()
logger.info(f'there are {prep_data_df.shape[0]} records after preprocessing')
# Convert last_review to datetime
prep_data_df['last_review'] = pd.to_datetime(prep_data_df['last_review'])

INFO:__main__:there are 19001 records after preprocessing


In [13]:
prep_data_profile = pandas_profiling.ProfileReport(prep_data_df)
prep_data_profile.to_widgets()

Summarize dataset:   0%|          | 0/30 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

### End run

In [14]:
run.finish()